In [1]:
import os
import glob
import json
import shutil
from multiprocessing import Pool

In [2]:
! pip install fire

     |████████████████████████████████| 92kB 3.5MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115928 sha256=7768888670818e2f6b925fe97e83099f3c47e9aabfc5dcc5b4875a8310559b4d
  Stored in directory: /root/.cache/pip/wheels/af/19/30/1ea0cad502dcb4e66ed5a690279628c827aea38bbbab75d5ed
Successfully built fire


In [3]:
! pip install easyocr

     |████████████████████████████████| 63.0MB 70kB/s 


In [4]:
import fire
import easyocr
import numpy as np
import torch

In [6]:
from PIL import Image
from skimage import transform
from skimage.feature import canny
from skimage.color import rgb2gray, gray2rgb

In [7]:
def multi_boxes_mask(image, boxes, pad_crop=5):
    """
    image: np.uint8 (h, w, c)
    boxes: np.int32 (n, 4) ymin, xmin, ymax, xmax
    """
    image = image.copy()
    mask = np.zeros_like(image)
    ih, iw, ic = image.shape
    resize = lambda a, b: transform.resize(a, b, preserve_range=True).astype(np.uint8)
    import matplotlib.pyplot as plt
    
    for box in boxes:
        # image[box[0]: box[2], box[1]: box[3], :] = 0
        box[:2] = np.maximum(box[:2] - pad_crop, 0)
        box[2:] = np.minimum(box[2:] + pad_crop, image.shape[:2])
        
        patch = image[box[0]: box[2], box[1]: box[3], :]
        pure_white = (patch > 253).all(axis=-1).astype(np.uint8)
        mask[box[0]: box[2], box[1]: box[3], :] = pure_white[..., None]
        
        # plt.subplot(2, 1, 1)
        # plt.imshow(patch)
        # plt.subplot(2, 1, 2)
        # plt.imshow(pure_white)
        # plt.colorbar()
        # plt.show()
        
        print('pure_white ', pure_white.sum())
    
    shift = 3
    shifts = [
        (0, 0), (shift, 0), (-shift, 0), (0, shift), (0, -shift),
        (shift, shift), (-shift, shift), (shift, -shift), (-shift, -shift)
    ]
    # shifts = []
    for offset in shifts:
        ox, oy = offset
        _mask = mask.copy()

        slice_y = slice(max(0, 0 + oy), min(ih, ih + oy))
        slice_x = slice(max(0, 0 + ox), min(iw, iw + ox))
        print(slice_y, slice_x)
        _mask = _mask[
            max(0, 0 + oy): min(ih, ih + oy),
            max(0, 0 + ox): min(iw, iw + ox),
            :
        ]
        crop_pad = [
            (max(0, -oy), max(0, oy)),
            (max(0, -ox), max(0, ox)),
            (0, 0)
        ]
        _mask = np.pad(_mask, crop_pad)
        print(
            crop_pad,
            np.abs(_mask - mask).sum(),
            np.abs(mask - np.clip(_mask + mask, 0, 1)).sum()
        )
        mask = np.clip(_mask + mask, 0, 1)

    image = image * (1 - mask) + mask * 255 * 0
    mask *= 255
    return image, mask


In [8]:
def cast_pred_type(pred):
    result = []
    for tup in pred:
        coord, txt, score = tup
        coord = np.array(coord).tolist()
        score = float(score)
        result.append((coord, txt, score))
    return result


In [9]:
def detect(root_dir):
    reader = easyocr.Reader(['en'])
    image_dir = os.path.join(root_dir, 'img')
    images = glob.glob(os.path.join(image_dir, '*.png'))
    images += glob.glob(os.path.join(image_dir, '**', '*.png'))
    # images = images[:3]
    #assert len(images) > 9000

    out_json = os.path.join(root_dir, 'ocr.json')
    out_anno = {}
    print(f"Find {len(images)} images!")

    for i, image_path in enumerate(images):
        print(F"{i}/{len(images)}")
        img_name = os.path.basename(image_path)
        pred = reader.readtext(image_path)
        out_anno[img_name] = cast_pred_type(pred)

    with open(out_json, 'w') as f:
        json.dump(out_anno, f)

In [10]:
def point_to_box(anno_json):
    with open(anno_json, 'r') as f:
        ocr_anno = json.load(f)
    
    boxed_anno = {}
    for k, v in ocr_anno.items():
        img_ocr_infos = []
        for txt_info in v:
            coord, txt, score = txt_info
            xmin = min([p[0] for p in coord])
            xmax = max([p[0] for p in coord])
            ymin = min([p[1] for p in coord])
            ymax = max([p[1] for p in coord])
            box = [xmin, ymin, xmax, ymax]
            img_ocr_infos.append([box, txt, score])
        boxed_anno[k] = img_ocr_infos

    
    print(anno_json)
    out_path = str(anno_json).replace('json','box.json')
    with open(out_path, 'w') as f:
        json.dump(boxed_anno, f)

In [11]:
def _mask_white_txt(args):
    img_name, img_boxes, img_dir, out_dir = args
    img_path = os.path.join(img_dir, img_name)
    out_path = os.path.join(out_dir, img_name)
    
    if os.path.exists(out_path):
        return
    # if img_name != '01487.png':
    #     continue
    
    print(out_path)
    img_boxes = [box_info[0] for box_info in img_boxes]
    if len(img_boxes) > 0:
        boxes = np.asarray(img_boxes, dtype=np.int32)
        # print(boxes)
        boxes = np.concatenate([boxes[:, ::-1][:, 2:], boxes[:,::-1][:, :2]], axis=1)
        # print(boxes)
        # x,y,x,y -> y,x,y,x
        img = np.array(Image.open(img_path).convert('RGB'))
        # res = inpaint_model.inpaint_multi_boxes(img, boxes)
        masked_img, mask = multi_boxes_mask(img, boxes)

        Image.fromarray(masked_img).save(out_path)
        out_path = os.path.join(out_dir, img_name.replace('.png', '.mask.png'))
        Image.fromarray(mask).save(out_path)
    else:
        img = np.asarray(Image.open(img_path).convert('RGB'))
        shutil.copy(img_path, out_path)

        mask = np.zeros_like(img)
        out_path = os.path.join(out_dir, img_name.replace('.png', '.mask.png'))
        Image.fromarray(mask).save(out_path)


In [12]:
def generate_mask(ocr_box_anno, img_dir, out_dir):
    os.makedirs(out_dir, exist_ok=True)
    with open(ocr_box_anno, 'r') as f:
        boxes_anno = json.load(f)

    # for i, (img_name, img_boxes) in enumerate(boxes_anno.items()):
    #     pass
    
    with Pool(16) as pool:
        args = [
            (img_name, img_boxes, img_dir, out_dir)
            for img_name, img_boxes in boxes_anno.items()
        ]
        pool.map(_mask_white_txt, args)


In [28]:
if __name__ == "__main__":
    """
    detect -[ocr.json]-> point_to_box -[ocr.box.json]->  generate_mask
    """
    # detect()
    # point_to_box('/home/ron/Downloads/hateful_meme_data/ocr.json')
    # print('hi')
    # generate_mask(
    #     '/home/ron/Downloads/hateful_meme_data/ocr.box.json',
    #     '/home/ron/Downloads/hateful_meme_data_phase2/img',
    #     '/home/ron/Downloads/hateful_meme_data_phase2/img_mask_3px'
    # )

    fire.Fire({
        "detect": detect,
        "point_to_box": point_to_box,
        "generate_mask": generate_mask,
    })

ERROR: Cannot find key: -f
Usage: ipykernel_launcher.py <command>
  available commands:    detect | point_to_box | generate_mask

For detailed information on this command, run:
  ipykernel_launcher.py --help


FireExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
from pathlib import Path

In [20]:
data_dir = Path.cwd().parent /'content/drive/MyDrive/hateful-memes'

img_tar_path = data_dir / "img"
train_path = data_dir / "train.jsonl"
dev_path = data_dir / "dev.jsonl"
test_path = data_dir / "test.jsonl"
ocr_path = data_dir/ "ocr.json"
ocr_box_path = data_dir/ "ocr.box.json"
output = data_dir / "output"


In [ ]:
detect(data_dir)

In [58]:
point_to_box(ocr_box_path)

/content/drive/MyDrive/hateful-memes/ocr.json


In [21]:
generate_mask(ocr_box_path,img_tar_path,output)

Выходные данные были обрезаны до нескольких последних строк (5000).
slice(0, 400, None) slice(0, 316, None)
/content/drive/MyDrive/hateful-memes/output/23845.png
slice(0, 764, None) slice(0, 509, None)
[(0, 3), (0, 3), (0, 0)] 4935936 4916655
[(0, 0), (0, 0), (0, 0)] 0 0
slice(3, 531, None) slice(0, 797, None)
slice(0, 400, None) slice(3, 316, None)
[(0, 3), (0, 3), (0, 0)] 2243304 1838295
slice(3, 800, None) slice(0, 407, None)
[(0, 0), (0, 3), (0, 0)] 2323200 2314125
slice(0, 400, None) slice(0, 313, None)
[(3, 0), (3, 0), (0, 0)] 3201417 3093660
[(0, 0), (3, 0), (0, 0)] 1628928 1622565
slice(3, 400, None) slice(0, 316, None)
[(0, 3), (3, 0), (0, 0)] 2181504 1685295
[(0, 3), (3, 0), (0, 0)] 5077248 5057415
slice(0, 528, None) slice(3, 800, None)
slice(0, 797, None) slice(3, 410, None)
[(0, 3), (0, 0), (0, 0)] 1267200 1262250
slice(0, 397, None) slice(0, 316, None)
[(3, 0), (0, 3), (0, 0)] 2025984 2018070
[(3, 0), (0, 0), (0, 0)] 1142784 1138320
/content/drive/MyDrive/hateful-memes/ou